In [27]:
import pandas as pd
import numpy as np
import string
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import CountVectorizer

# Read the Data

In [2]:
data = pd.read_csv('Amazon_Unlocked_Mobile.csv')
data.head()

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I feel so LUCKY to have found this used (phone...,1.0
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,"nice phone, nice up grade from my pantach revu...",0.0
2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,Very pleased,0.0
3,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,It works good but it goes slow sometimes but i...,0.0
4,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,Great phone to replace my lost phone. The only...,0.0


# Word count in each review

In [3]:
#Number of words
data['word_count'] = data['Reviews'].apply(lambda x: len(str(x).split(" ")))
data[['Reviews','word_count']].head()

,Reviews,word_count
0,I feel so LUCKY to have found this used (phone...,72
1,"nice phone, nice up grade from my pantach revu...",40
2,Very pleased,2
3,It works good but it goes slow sometimes but i...,17
4,Great phone to replace my lost phone. The only...,43


In [4]:
data_new1 = data[data['Reviews'].notnull()]

# Data Pre-processing

In [5]:
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import re, string
from nltk import pos_tag
import nltk
from nltk.tokenize import word_tokenize
tweets_texts = data_new1["Reviews"].tolist()
stopwords=stopwords.words('english')
english_vocab = set(w.lower() for w in nltk.corpus.words.words())

In [6]:
def process_tweet_text(tweet):
    tweet = re.sub(r'\$\w*','',tweet) # Remove tickers
    tweet = re.sub(r'https?:\/\/.*\/\w*','',tweet) # Remove hyperlinks
    tweet = re.sub(r'['+string.punctuation+']+', ' ',tweet) # Remove puncutations like 's
    #tokens = [i.lower() for i in tweet if i not in stopwords and len(i) > 2 and  i in english_vocab]
    return tweet

In [7]:
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
stopwords=set(stopwords.words('english'))
wnl = WordNetLemmatizer()

def penn2morphy(penntag):
    """ Converts Penn Treebank tags to WordNet. """
    morphy_tag = {'NN':'n', 'JJ':'a',
                  'VB':'v', 'RB':'r'}
    try:
        return morphy_tag[penntag[:2]]
    except:
        return 'n' 
    
def lemmatize_sent(text): 
    # Text input is string, returns lowercased strings.
    text = process_tweet_text(text)
    
    tokenized_list = [w for w in word_tokenize(text)]
    l1=[]
    for i in tokenized_list:
        if i.lower() not in stopwords:
            l1.append(i)
        else:
            pass
    return [wnl.lemmatize(word.lower(), pos=penn2morphy(tag)) 
            for word, tag in pos_tag(l1)]

#lemmatize_sent('I feel so LUCKY to have found this used (phone)')

In [8]:
# words = []
# for tw in tweets_texts:
#       words += lemmatize_sent(tw)

# Processed Data

In [9]:
data_new1['Processed_data'] = data_new1['Reviews'].apply(lambda x: lemmatize_sent(x))

/usr/local/lib/python3.4/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [10]:
data_new1['Processed_data'].head(10)

0    [feel, lucky, find, use, phone, u, use, hard, ...
1    [nice, phone, nice, grade, pantach, revue, cle...
2                                            [pleased]
3    [work, good, go, slow, sometimes, good, phone,...
4    [great, phone, replace, lose, phone, thing, vo...
5    [already, phone, problem, know, state, used, d...
6    [charge, port, loose, get, solder, need, new, ...
7    [phone, look, good, stay, charge, buy, new, ba...
8    [originally, use, samsung, s2, galaxy, sprint,...
9    [battery, life, great, responsive, touch, issu...
Name: Processed_data, dtype: object

In [11]:
# from collections import Counter
# sorted(Counter(words).most_common(15))

# Sentiment Analysis

In [12]:
data_new1['Sentiments'] = 'Neutral'
data_new1['Sentiments'][data_new1['Rating']==3] = 'Neutral'
data_new1['Sentiments'][data_new1['Rating']<3] = 'Negative'
data_new1['Sentiments'][data_new1['Rating']>3] = 'Positive'

/usr/local/lib/python3.4/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.4/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib64/python3.4/site-packages/pandas/core/generic.py:4702: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/usr/local/lib/py

In [13]:
data_new1['Processed_data_strings'] = data_new1['Processed_data'].apply(lambda x: ' '.join(x))

/usr/local/lib/python3.4/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [14]:
df = pd.DataFrame({"Processed_data_strings": data_new1['Processed_data_strings'],"Sentiments":data_new1['Sentiments']})

In [15]:
df.head()

,Processed_data_strings,Sentiments
0,feel lucky find use phone u use hard phone lin...,Positive
1,nice phone nice grade pantach revue clean set ...,Positive
2,pleased,Positive
3,work good go slow sometimes good phone love,Positive
4,great phone replace lose phone thing volume bu...,Positive


In [16]:
df = df[df['Processed_data_strings'].notnull()]

# Dividing into train-test split

In [17]:
# split the dataset into training and validation datasets 

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df['Processed_data_strings'],df['Sentiments'])

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)


print('Number of rows in the total set: {}'.format(df.shape[0]))
print('Number of rows in the training set: {}'.format(train_x.shape[0]))
print('Number of rows in the test set: {}'.format(valid_x.shape[0]))

Number of rows in the total set: 413778
Number of rows in the training set: 310333
Number of rows in the test set: 103445


# Count Vectorizer

In [18]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(df['Processed_data_strings'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [19]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
        
    return metrics.accuracy_score(predictions, valid_y)

# Naive Bayes 

In [20]:
# Naive Bayes on Count Vectors
nb=naive_bayes.MultinomialNB()
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
print("NB, Count Vectors: ", accuracy)

NB, Count Vectors:  0.8489728841413311


In [21]:
nb.fit(xtrain_count, train_y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [22]:
y_predicted = nb.predict(xvalid_count)

In [23]:
from sklearn.metrics import accuracy_score
accuracy_score(valid_y, y_predicted) 

0.8489728841413311

In [24]:
for i in range(len(y_predicted)):
    print(y_predicted[i], " ", valid_y[i])

2   2
2   2
2   2
2   2
2   2
2   0
0   0
2   2
2   2
0   0
0   2
2   2
0   0
0   1
2   2
2   2
2   2
0   0
2   2
2   2
2   2
2   2
2   2
2   2
0   2
2   0
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
1   2
0   0
0   0
1   2
2   2
2   2
2   2
1   2
0   0
2   2
2   2
2   2
0   0
2   2
2   2
2   2
0   0
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
0   0
0   0
1   2
0   0
2   2
0   0
2   2
2   2
0   0
2   2
0   0
2   2
2   2
2   2
2   2
2   2
2   2
2   1
0   0
0   0
2   2
1   1
2   2
2   2
2   2
2   2
2   2
0   0
2   2
2   2
2   2
2   2
2   0
0   0
0   0
2   2
2   2
0   0
2   2
0   0
0   0
2   2
2   2
2   0
2   2
0   0
2   2
2   2
2   2
0   0
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   1
2   2
1   2
2   2
0   0
2   2
2   2
2   2
2   2
0   0
2   1
2   2
0   0
2   2
2   2
0   0
2   2
2   2
0   0
2   2
2   2
2   2
2   2
2   2
2   2
2   2
0   0
2   2
1   1
0   0
2   0
2   2
2   2
2   2
2   2
2   0
0   0
1   1
0   1
2   2
0   0
0   0
2   

2   2
2   2
0   0
0   1
2   2
2   2
2   2
2   0
2   2
2   2
2   2
0   0
2   2
2   2
2   2
1   1
2   2
0   0
2   2
2   2
0   0
2   2
2   2
2   2
2   2
0   1
2   2
2   2
2   2
2   2
2   1
2   0
0   1
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
0   0
2   2
2   2
0   0
2   2
2   2
1   1
2   2
2   1
0   2
2   2
2   2
2   2
2   2
0   0
2   2
2   2
2   2
0   0
0   0
2   2
2   2
2   2
2   2
2   2
2   2
2   2
0   0
2   2
2   2
0   2
2   2
2   2
2   2
2   2
0   1
2   0
0   0
2   2
2   2
2   2
2   2
2   2
0   0
1   1
0   0
2   2
2   2
2   2
0   0
2   2
2   2
2   2
0   0
2   2
2   2
2   1
2   2
0   0
2   2
2   0
2   2
0   0
2   2
2   2
0   0
2   2
2   2
2   2
2   2
2   2
0   0
2   2
2   2
2   2
2   0
0   0
2   2
2   2
2   2
2   2
2   2
2   2
0   0
1   1
2   2
2   1
2   2
2   2
2   2
2   2
2   2
2   2
1   0
0   0
0   0
2   2
2   2
0   0
2   2
2   2
2   0
2   2
2   2
2   2
2   2
2   2
2   2
2   2
0   0
2   1
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
0   0
2   2
2   1
2   2
2   2
2   

0   0
2   2
0   0
0   0
2   2
2   2
0   0
2   2
2   2
2   2
2   2
2   2
2   2
0   1
2   2
2   2
2   2
2   2
2   2
0   0
2   2
0   0
2   0
2   2
2   2
0   0
0   0
0   0
2   2
2   2
2   2
0   0
0   1
2   2
2   1
2   2
2   2
2   2
1   2
2   0
0   0
2   2
2   0
0   0
2   2
2   2
2   2
2   2
2   2
0   0
2   2
2   2
0   0
2   2
2   2
0   0
2   2
2   2
2   2
2   2
2   0
0   2
2   2
2   2
2   2
2   2
2   2
0   2
0   0
2   2
2   2
2   2
2   2
2   2
2   2
0   0
2   2
2   2
2   2
2   0
2   2
0   0
2   2
2   1
2   2
2   1
2   2
2   2
2   2
0   0
0   0
2   2
0   0
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
0   0
2   2
0   0
2   2
2   2
0   0
2   2
0   0
0   0
0   1
2   2
2   2
2   2
2   2
2   2
2   2
2   2
0   2
2   2
0   0
0   0
0   0
2   2
2   2
2   2
2   2
0   2
2   2
2   2
2   2
2   2
0   2
2   0
2   2
0   1
0   0
0   0
2   2
2   2
2   2
2   2
0   0
0   0
0   0
0   0
2   2
0   0
2   1
2   2
2   2
0   0
2   2
2   2
2   2
2   2
0   2
1   0
2   2
2   2
2   2
2   2
2   2
2   2
2   2
0   

2   2
2   2
2   2
0   0
2   2
2   2
2   2
2   2
0   0
0   2
2   1
2   2
0   0
2   2
2   2
2   2
0   0
2   2
2   2
2   2
0   0
2   2
2   1
2   2
2   2
0   2
2   2
2   2
2   2
2   2
0   0
2   2
0   0
2   2
2   2
2   2
2   2
2   2
1   1
2   2
0   2
0   0
2   2
0   0
2   2
0   0
2   2
2   2
2   2
2   2
2   2
0   1
2   2
2   2
0   0
2   2
2   2
1   1
2   2
2   2
2   2
2   2
2   0
2   2
2   2
2   2
2   2
2   2
2   0
2   2
0   0
2   2
2   2
2   1
1   1
2   2
0   0
2   2
2   2
2   2
2   2
2   2
2   2
0   2
0   0
2   0
2   2
0   0
2   2
0   2
2   2
2   2
2   2
0   0
2   2
2   2
2   2
2   2
2   2
0   0
0   0
2   2
0   2
2   2
2   2
2   2
0   1
2   2
2   2
2   2
1   1
0   1
0   0
2   2
2   2
2   2
0   0
2   2
2   2
2   2
2   2
2   2
0   0
0   0
2   2
2   2
2   1
2   2
1   1
2   2
0   0
0   0
0   0
2   2
2   2
2   1
2   2
2   2
2   2
2   2
2   0
0   0
2   2
0   0
2   2
2   2
0   1
2   2
2   2
2   2
2   2
1   0
0   0
2   0
2   2
0   0
2   1
2   2
2   2
2   2
2   2
2   2
2   2
0   0
2   2
2   2
2   

2   2
1   0
2   2
2   2
2   2
0   0
2   2
2   2
2   2
2   2
0   0
0   0
2   2
2   2
2   2
2   2
2   2
2   2
2   2
0   0
2   2
1   1
0   0
2   2
0   0
2   2
2   2
0   2
2   2
2   2
2   2
1   2
2   2
0   1
2   2
2   1
0   2
0   0
0   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
0   0
2   2
2   2
2   2
2   2
0   0
2   2
2   1
1   1
2   2
2   2
2   2
2   2
0   0
2   2
2   2
0   0
2   2
2   2
2   2
0   1
2   2
2   2
2   2
2   2
1   1
2   2
2   2
2   2
2   2
2   2
0   2
2   2
2   2
2   1
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
0   2
0   0
2   2
2   2
2   2
2   2
0   1
2   2
2   2
2   2
2   0
2   2
1   2
2   2
2   2
0   0
2   2
0   0
2   2
2   2
2   2
2   1
2   2
2   2
2   2
2   2
2   2
2   2
2   1
2   2
2   2
2   2
2   2
0   0
2   2
2   2
2   2
2   2
2   2
2   2
0   1
0   0
2   2
2   2
0   0
0   0
2   2
2   2
2   2
2   2
2   1
2   2
2   2
2   2
0   0
2   2
0   0
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
0   0
2   2
0   0
0   0
2   2
2   2
2   2
2   1
2   2
2   2
0   1
2   

...

0   0
2   2
2   2
2   2
2   2
2   2
2   2
2   2
0   1
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   0
2   2
2   2
0   0
2   2
1   0
2   2
2   2
2   2
1   2
1   0
0   1
2   2
2   2
2   2
2   2
0   0
2   2
2   1
2   2
2   2
2   2
0   0
2   2
2   2
2   2
0   1
2   1
2   2
2   1
2   2
2   2
2   1
2   2
2   0
2   2
2   2
2   2
2   2
2   2
2   2
0   0
2   2
0   0
2   2
2   2
0   2
0   0
2   2
2   2
0   0
2   2
2   2
0   2
2   2
0   0
2   2
0   2
2   2
2   2
2   2
2   2
0   0
0   0
2   2
0   0
2   2
0   0
2   0
2   2
2   2
2   2
2   2
2   2
0   0
2   0
2   2
2   2
0   0
2   2
2   2
2   2
0   0
2   2
2   2
2   2
0   1
2   1
2   2
0   0
2   2
2   2
2   2
0   0
2   2
0   0
2   2
2   2
0   0
0   1
2   0
0   0
0   2
2   2
2   2
2   2
0   0
0   2
2   2
2   2
2   2
2   2
0   0
2   2
2   2
2   2
2   2
0   0
2   2
2   2
2   0
2   2
0   0
2   0
0   0
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   0
2   2
2   2
2   2
2   2
2   2
0   1
2   2
2   2
2   2
2   2
2   

2   2
2   2
2   2
2   2
0   0
0   0
2   1
2   2
2   2
2   2
2   0
2   2
0   0
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
0   0
0   0
2   1
0   0
2   2
2   2
2   2
2   2
0   0
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   0
1   2
2   2
2   2
0   0
2   1
0   0
2   1
1   0
2   2
0   0
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
1   2
2   2
2   2
2   2
0   0
2   2
0   0
0   0
2   0
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
0   0
2   2
0   1
2   2
0   1
0   0
2   0
2   2
2   2
2   2
2   2
2   2
2   0
0   0
2   0
2   2
2   1
2   2
2   2
2   2
0   0
2   2
2   0
2   2
0   0
2   2
0   0
2   2
0   0
2   2
2   2
2   2
2   2
2   2
2   2
0   0
2   0
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
1   0
2   2
2   2
0   0
2   2
2   2
2   2
2   2
0   0
2   2
2   2
2   2
2   2
2   2
2   2
0   1
2   2
2   2
2   2
0   1
0   0
0   0
2   2
2   2
2   2
2   2
0   0
2   2
2   2
2   2
2   2
1   0
2   2
2   2
2   2
0   2
0   2
0   0
0   0
2   2
0   

0   0
2   2
2   2
0   0
2   2
2   2
2   2
0   0
2   2
2   2
2   2
2   2
2   2
2   2
0   0
2   2
2   2
2   2
2   2
2   2
0   0
2   2
2   2
0   1
2   2
2   2
2   2
2   2
2   2
0   1
2   2
0   2
2   1
2   2
2   2
1   2
2   2
0   0
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   0
2   2
0   0
0   1
2   2
2   2
0   0
2   2
2   2
2   2
0   0
2   2
2   2
2   2
2   2
2   2
2   2
0   0
2   2
2   2
0   0
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   0
0   0
2   2
2   2
2   2
2   2
2   2
0   0
2   2
1   2
0   0
0   0
2   2
2   2
2   2
1   1
2   0
0   0
2   0
2   2
2   2
2   2
2   0
0   0
2   2
2   0
2   1
0   2
2   2
2   0
2   2
2   2
0   1
2   2
0   0
2   2
0   0
0   0
2   2
0   0
2   2
2   2
2   2
2   2
2   2
0   0
2   2
2   2
0   0
2   2
1   2
2   1
2   2
2   2
2   2
0   0
0   0
2   2
0   1
2   2
2   2
2   2
2   2
2   0
2   2
2   2
2   2
2   0
2   1
1   1
2   2
2   2
2   2
0   2
1   2
2   0
2   2
2   2
2   2
2   2
2   2
2   0
2   1
2   2
2   1
0   0
0   2
2   2
2   2
0   0
0   0
1   

2   2
2   2
2   2
2   2
2   2
0   0
2   2
2   2
0   2
2   2
2   2
2   2
2   2
0   0
0   0
2   2
0   2
2   2
2   2
0   0
2   1
2   0
2   2
2   2
2   2
2   2
2   2
0   1
2   2
2   2
0   0
2   0
0   0
0   0
0   0
2   2
2   2
2   2
2   2
2   2
2   2
0   0
2   2
2   2
2   1
2   2
0   0
2   2
2   2
2   2
0   0
2   2
2   2
0   0
0   0
2   0
0   0
2   2
2   2
2   2
2   0
2   0
2   2
2   2
0   0
0   0
2   2
2   2
2   0
2   2
0   0
2   2
2   2
2   2
1   2
2   2
2   2
2   2
2   1
2   2
2   0
2   2
2   2
2   1
2   2
2   0
2   0
0   0
0   0
2   2
2   2
2   2
2   2
2   2
0   0
2   2
2   2
2   2
2   2
0   2
2   2
2   1
2   2
2   2
2   0
2   2
2   2
2   2
0   0
2   2
2   2
2   2
2   2
2   2
2   0
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   0
0   1
2   2
2   1
2   2
2   2
2   0
2   2
2   2
2   2
2   2
0   1
2   2
2   2
1   0
2   2
0   0
0   0
2   2
2   2
2   2
1   0
1   2
1   1
2   2
2   2
2   2
0   0
2   2
2   2
2   2
2   0
2   2
2   0
0   1
2   2
2   2
0   1
2   2
2   2
1   2
2   

2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
0   0
2   2
2   2
2   2
2   2
0   0
2   2
2   1
2   2
0   1
0   0
0   0
2   2
0   0
1   0
2   2
0   0
2   0
2   2
2   2
2   2
1   2
2   2
0   0
2   2
0   0
2   2
1   1
2   2
2   2
2   2
2   0
2   2
2   2
2   2
2   1
2   2
0   0
2   2
2   2
2   2
0   0
1   1
0   2
0   1
0   0
2   2
2   2
2   2
2   1
2   2
2   2
0   0
2   2
2   2
2   2
2   2
2   2
0   0
2   2
2   2
2   2
2   2
0   0
2   2
2   1
2   2
2   0
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
0   0
0   0
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
0   1
2   2
2   2
0   0
2   2
2   2
0   0
1   0
2   2
2   2
2   2
0   0
2   2
2   2
2   2
2   2
2   2
2   2
0   0
2   2
0   0
0   0
0   0
2   2
2   2
0   0
2   2
0   0
2   2
2   1
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
2   2
0   0
0   0
2   2
2   2
2   2
2   2
0   0
2   2
2   2
2   0
0   0
0   0
2   2
2   2
2   2
0   0
2   2
2   

In [ ]:
#Implementing SVM model

# SVM on Ngram Level TF IDF Vectors
accuracy = train_model(svm.SVC(), xtrain_count, train_y, xvalid_count)
print("SVM, Count Vectors: ", accuracy)

In [ ]:
# Extereme Gradient Boosting on Count Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_count.tocsc(), train_y, xvalid_count.tocsc())
print("Xgb, Count Vectors: ", accuracy)